In [2]:
import numpy as np
import tifffile
from scipy import ndimage
from skimage.filters import threshold_otsu
import matplotlib.pyplot as plt
from skimage.measure import label, regionprops


file_name = f"/Users/demir/Documents/Hva_AI/afstudeerproject/codes_yunus/input/tif_deneme_1.tif"

img = tifffile.imread(file_name)  # (Z, C, Y, X)
print(img.shape)

# Sadece 4. kanalı (C3, index 3) tüm Z-stack ile al
channel_index = 3
c3_stack = img[:, channel_index, :, :]  # Shape: (9, 1024, 1024)

print(c3_stack.shape)


(9, 4, 1024, 1024)
(9, 1024, 1024)


In [4]:
from cellpose import models, io
import matplotlib.pyplot as plt

# Cellpose modelini yükle
model = models.Cellpose(model_type='cyto')  # veya 'nuclei' hücre çekirdeği için

# normalize et (Cellpose genellikle [0,1] veya [0,255] bekler)
stack_normalized = (c3_stack - np.min(c3_stack)) / (np.max(c3_stack) - np.min(c3_stack))
stack_scaled = (stack_normalized * 255).astype(np.uint8)  # dtype olarak uint8 daha güvenli olur

# Modeli çalıştır (3D segmentasyon)
masks, flows, styles, diams = model.eval(
    stack_scaled,
    channels=[0, 0],  # tek kanal kullanılıyor
    do_3D=True,
    diameter=None,   # otomatik çap tahmini
    min_size=100     # çok küçük segmentleri filtrele
)

# Sonuçları görselleştir
z_index = len(masks) // 2  # ortadaki slice
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.imshow(stack_scaled[z_index], cmap='gray')
plt.title('Orijinal')

plt.subplot(1, 2, 2)
plt.imshow(masks[z_index], cmap='nipy_spectral')
plt.title('Cellpose 3D Segmentasyon (Z-slice)')
plt.show()


could not estimate diameter, does not work on non-2D images
z_axis not specified, assuming it is dim 0
if this is actually the channel_axis, use 'model.eval(channel_axis=0, ...)'


In [ ]:
# Maske sonuçlarını kaydet (.tif formatında)
output_mask_path = "/Users/demir/Documents/Hva_AI/afstudeerproject/codes_yunus/output/cellpose_mask_c3.tif"
tifffile.imwrite(output_mask_path, masks.astype(np.uint16))
print(f"Maskeler kaydedildi: {output_mask_path}")
